# Automated ML

Needed dependencies imported here.

In [2]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
import joblib


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code CQ8VXT79N to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
capstone-project
capstone-project
francecentral
2d4b3a3e-de2a-45bb-9ac0-29caf8f98da4


## Create an Azure ML experiment

An Experiments named 'titanic-survival' and a folder to hold the training scripts are created. The script runs will be recorded under the experiment in Azure.

In [4]:
# choose a name for experiment
experiment_name = 'titanic-survival'
project_folder = './capstone-project'

experiment=Experiment(ws, experiment_name)
experiment
run = experiment.start_logging()

In [5]:
# Name for the cluster
cpu_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    # Poll for a minimum number of nodes (min_nodes = 1). 
    # If no min node count is provided it uses the scale settings for the cluster.
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2', min_nodes=1, max_nodes=6)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Creating a new compute cluster...
Creating
Succeeded.....................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 1, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-02-20T07:30:50.823000+00:00', 'errors': None, 'creationTime': '2021-02-20T07:28:51.863668+00:00', 'modifiedTime': '2021-02-20T07:29:07.310571+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 6, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS3_V2'}


## Dataset

Dataset used for this project is "Titanic - Machine Learning from Disaster" dataset from [Kaggle](https://www.kaggle.com/c/titanic). Aim here is to predicts which passengers survived the Titanic shipwreck.

Dataset features:

| Variable | Definition | Key |
|----------|------------|-----|
|survival|Survival|0 = No, 1 = Yes
|pclass|Ticket class|1 = 1st, 2 = 2nd, 3 = 3rd
|sex|Sex|	
|Age|Age in years|	
|sibsp|# of siblings / spouses aboard the Titanic|	
|parch|# of parents / children aboard the Titanic|	
|ticket|Ticket number|	
|cabin|Cabin number|	
|fare|Passenger fare|	
|embarked|Port of Embarkation|C = Cherbourg, Q = Queenstown, S = Southampton

In [10]:
key = "titanic-survival-data"
description_text = "Dataset for Titanic disaster survival with passenger info"

# retrieve data from workspace
if key in ws.datasets.keys(): 
        dataset = ws.datasets[key] 

else:
        # If not found, create AutoML Dataset and register it into Workspace
        my_dataset = 'https://raw.githubusercontent.com/j0h4nnesk/Capstone_project_Titanic_Survival/main/train.csv'
        dataset = Dataset.Tabular.from_delimited_files(my_dataset)        
        # Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

df = dataset.to_pandas_dataframe()
df.describe()

# Data columns
x = df.drop('Survived',axis=1)
y = df[['Survived']]

## AutoML Configuration

Here is an overview of AutoML settings and configurations:

##### AutoML settings
*n_cross_validations=2:* How many cross validations to perform when user validation data is not specified.

*primary_metric='accuracy':* The metric that Automated Machine Learning will optimize for model selection.

*enable_early_stopping=True* Whether to enable early termination if the score is not improving in the short term. 

*max_concurrent_iterations=4:* Represents the maximum number of iterations that would be executed in parallel.

*experiment_timeout_minutes=30:* Exit criteria that is used to define how long, in minutes, the experiment should continue to run. To help avoid experiment time out failures, 30 minutes was used as the timeout value.

*verbosity=logging.INFO:* The verbosity level for writing to the log file.

##### AutoML config

*compute_target=compute_target:* The compute target to run the Automated Machine Learning experiment on.

*task='classification':* The type of task to run.

*training_data=dataset:* The training data to be used within the experiment. It should contain both training features and a label column.

*label_column_name='Survived':* The name of the label column, the target column based on which the prediction is done.

*path = project_folder:* The full path to the Azure Machine Learning project folder.

*featurization= 'auto':* 'auto' / 'off' / FeaturizationConfig Indicator for whether featurization step should be done automatically or not.

*debug_log = "automl_errors.log":* The log file to write debug information to. 

*enable_onnx_compatible_models=False:* Whether to enable or disable enforcing the ONNX-compatible models. (([ONNX](https://docs.microsoft.com/en-us/azure/machine-learning/concept-onnx)) can help optimize the inference of your machine learning model. Inference, or model scoring, is the phase where the deployed model is used for prediction, most commonly on production data.)

In [12]:

# Automl settings
automl_settings = {"n_cross_validations": 2,
                    "primary_metric": 'accuracy',
                    "enable_early_stopping": True,
                    "max_concurrent_iterations": 4,
                    "experiment_timeout_minutes": 30,
                    "verbosity": logging.INFO
                    }

# Parameters for AutoMLConfig
automl_config = AutoMLConfig(compute_target = compute_target,
                            task='classification',
                            training_data=dataset,
                            label_column_name='Survived',
                            path = project_folder,
                            featurization= 'auto',
                            debug_log = "automl_errors.log",
                            enable_onnx_compatible_models=False,
                            **automl_settings
                            )

In [13]:
remote_run = experiment.submit(automl_config, show_output = True)
remote_run.wait_for_completion()

Running on remote.
No run_configuration provided, running on compute-cluster with default configuration
Running on remote compute: compute-cluster
Parent Run ID: AutoML_8f0ce006-013e-4106-acfb-944126863f36

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to cu

{'runId': 'AutoML_8f0ce006-013e-4106-acfb-944126863f36',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-20T07:53:16.365519Z',
 'endTimeUtc': '2021-02-20T08:32:47.701767Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"titanic-survival","subscription_id":"2d4b3a3e-de2a-45bb-9ac0-29caf8f98da4","resource_group":"capstone-project","workspace_name":"capstone-project","region":"francecentral","compute_target":"compute-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":2,"y_min":null,"y_max":null,"num_classes":null,

## Run Details

Here `RunDetails` widget is used to show children runs of the experiment.

In [14]:
RunDetails(remote_run).show()

# Get details from each run
for child_run in remote_run.get_children():
    print('**************************************************')
    print(child_run)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

**************************************************
Run(Experiment: titanic-survival,
Id: AutoML_8f0ce006-013e-4106-acfb-944126863f36_34,
Type: azureml.scriptrun,
Status: Completed)
**************************************************
Run(Experiment: titanic-survival,
Id: AutoML_8f0ce006-013e-4106-acfb-944126863f36_33,
Type: azureml.scriptrun,
Status: Completed)
**************************************************
Run(Experiment: titanic-survival,
Id: AutoML_8f0ce006-013e-4106-acfb-944126863f36_32,
Type: azureml.scriptrun,
Status: Canceled)
**************************************************
Run(Experiment: titanic-survival,
Id: AutoML_8f0ce006-013e-4106-acfb-944126863f36_29,
Type: azureml.scriptrun,
Status: Completed)
**************************************************
Run(Experiment: titanic-survival,
Id: AutoML_8f0ce006-013e-4106-acfb-944126863f36_31,
Type: azureml.scriptrun,
Status: Canceled)
**************************************************
Run(Experiment: titanic-survival,
Id: AutoML_8

## Best Model

The best model is fetched from the automl experiments and all the properties of the model are displayed.



In [15]:
best_run, fitted_model = remote_run.get_output()

# get_metrics(): returns the metrics
print("Best run metrics :",best_run.get_metrics())
print('**************************************************')

# get_details(): returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())
print('**************************************************')

# get_properties(): fetches the latest properties of the run from the service
print("Best run properties :",best_run.get_properties())
print('**************************************************')

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Best run metrics : {'matthews_correlation': 0.6302355297288423, 'recall_score_micro': 0.8249156043734569, 'recall_score_weighted': 0.8249156043734569, 'f1_score_micro': 0.8249156043734569, 'average_precision_score_weighted': 0.8558629605200037, 'log_loss': 0.4667500930034354, 'AUC_micro': 0.8795093878205049, 'weighted_accuracy': 0.8405858644040407, 'precision_score_micro': 0.8249156043734569, 'recall_score_macro': 0.8076680749275977, 'f1_score_weighted': 0.8228346966654378, 'AUC_macro': 0.8685837387026218, 'f1_score_macro': 0.8108463468943486, 'AUC_weighted': 0.868583738702622, 'precision_score_weighted': 0.8290556719426578, 'precision_score_macro': 0.8230781677325676, 'average_precision_score_macro': 0.8518670188770152, 'norm_macro_recall': 0.6153361498551955, 'balanced_accuracy': 0.8076680749275977, 'accuracy': 0.8249156043734569, 'average_precision_score_micro': 0.8495296367064447, 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_8f0ce006-013e-4106-acfb-944126863f36_33/

In [16]:
best_run.get_file_names()

# Download the yaml file that includes the environment dependencies
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'env.yml')
# Download the model file

best_run.download_file('outputs/model.pkl', 'Automl_model.pkl')

In [17]:
best_run


Experiment,Id,Type,Status,Details Page,Docs Page
titanic-survival,AutoML_8f0ce006-013e-4106-acfb-944126863f36_33,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [18]:
best_run.register_model(model_name = "best_run_automl.pkl", model_path = './outputs/')

print(best_run)

Run(Experiment: titanic-survival,
Id: AutoML_8f0ce006-013e-4106-acfb-944126863f36_33,
Type: azureml.scriptrun,
Status: Completed)


## Model Deployment

As the best model from AutoML run has better accuracy than the one coming from the HyperDrive run, the AutoML model is deployed. Model is registered, deployed as a webservice and ingerence config is created.


In [21]:
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.aci.aciwebservice?view=azure-ml-py

model = remote_run.register_model(model_name = 'best_run_automl.pkl')
print(remote_run.model_id)


environment = best_run.get_environment()
entry_script='inference/scoring.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)

service_name = 'aciservice'

inference_config = InferenceConfig(entry_script = entry_script, environment = environment)

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(workspace = ws, name = service_name, models = [model], inference_config = inference_config, deployment_config = aci_config)
service.wait_for_deployment(show_output = True)

best_run_automl.pkl
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running................
Succeeded
ACI service creation operation finished, operation "Succeeded"


Request is sent to the webservice to test it.

In [25]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

data = {
    "data":
    [
        {
            'PassengerId': "892",
            'Pclass': "3",
            'Name': "Kelly, Mr. James",
            'Sex': "male",
            'Age': "34.5",
            'SibSp': "0",
            'Parch': "0",
            'Ticket': "330911",
            'Fare': "7.892",
            'Cabin': "",
            'Embarked': "Q",
        },
    ],
}

body = str.encode(json.dumps(data))

url = 'http://9c9c1d52-316e-45ed-b811-0bfb04b6a762.francecentral.azurecontainer.io/score'
api_key = 'zFb4t4JfleLdPkueQ03lWKC9Blu741FW' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

b'"{\\"result\\": [0]}"'


In [26]:
# Printing the logs
print(service.get_logs())

2021-02-20T12:07:22,495667900+00:00 - gunicorn/run 
2021-02-20T12:07:22,497972100+00:00 - rsyslog/run 
2021-02-20T12:07:22,505252900+00:00 - iot-server/run 
2021-02-20T12:07:22,536332100+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [27]:
# delete service 
service.delete()